In [ ]:
# !pip install tqdm
# conda install -c conda-forge osmnx

# Network Analysis

Network Analysis refers to a set of analytical techniques used to model and analyze the movement of goods, services, people, or information through a network of interconnected features such as roads, railways, pipelines, or utility lines. It enables users to solve complex spatial problems involving connectivity, accessibility, and optimal routing. <br>

Typically, the network consists of nodes (points) and edges (lines) that represent the connections between those points. Nodes can represent intersections, junctions, or any point of interest, while edges represent the paths or routes connecting those nodes. <br>

<div style="text-align: center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/2/2f/Small_Network.png" width="400">
</div>

Depends on the context, the network can be represented as various forms, such as graph, multigraph, or directed graph. <br>

<div style="text-align: center">
  <img src="./data/GraphTypes.jpg" width="1000">
</div>

* **(Undiredted)Graph**: A graph is a collection of nodes and edges, where each edge connects two nodes. Here, edges does not have directions, meaning the relationship between nodes is bidirectional. This is the most basic form of a network representation and is often used to model simple relationships between entities. <br>
* **Multigraph**: A multigraph is a type of graph that allows multiple edges between the same pair of nodes. This can be useful for representing complex relationships, such as different types of connections or interactions between nodes. <br>
* **Directed Graph**: A directed graph, or digraph, is a graph where the edges have a direction, indicating a one-way relationship between nodes. This can be useful for representing relationships where the direction matters, such as in transportation networks or information flow. <br>

This document uses the following two packages:

* <a href=https://networkx.org/documentation/stable/index.html>`NetworkX`</a> is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. <br>
* <a href=https://osmnx.readthedocs.io/en/stable/index.html>`OSMnx`</a> is a Python package that allows you to download and analyze street networks and other geospatial data from OpenStreetMap.</a> <br>


In [ ]:
# Import necessary libraries
import networkx as nx
import osmnx as ox

import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

## 1. Data Preparation
### 1.1. Load administrative boundaries

In [ ]:
# Load administrative boundaries (읍면동) data
emd_gdf = gpd.read_file('./data/emd_5179.geojson')
emd_gdf

In [ ]:
emd_gdf.explore()

### 1.2. Load Road Network

For the analysis, we will use a precomputed road network originally obtained from ViewT (https://viewt.ktdb.go.kr/) in the form of shapefiles.<br>
To utilize it in NetworkX and OSMnx, the shapefiles must be converted into a graph format. Details of this conversion process will be covered in the final section of this document.

In [ ]:
# Load road network data
# Original data is downloaded from View-T (https://viewt.ktdb.go.kr/)
G = ox.load_graphml('./data/road_network_seoul.graphml')
ox.plot_graph(G)

In [ ]:
# You can extract the edges and nodes of the network (graph) into GeoDataFrame with ox.graph_to_gdfs()
# It is not necessary for the network analysis, but it is useful for visualization and investigating the structure of the network
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
nodes.head(3)

In [ ]:
edges.head(3)

### 1.3. Find the nearest node to given locations

Given that the network analysis is only available for the nodes and edges of the road network, we need to find the nearest node to a given location. <br>
To do this, we can use the `nearest_nodes` function from OSMnx. This function takes a point (x and y coordinates) and returns the nearest node in the network. <br>

Syntax: 
```python
    nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT X COORDINATES`, `POINT Y COORDINATES`)
```

Source: https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.distance.nearest_nodes

In [ ]:
# Find the nearest nodes from the centroid of each administrative boundary
emd_gdf['nearest_nodes'] = ox.distance.nearest_nodes(G, emd_gdf.geometry.centroid.x, emd_gdf.geometry.centroid.y)
emd_gdf

## 2. Calculate the shortest path between two points

In this section, we will calculate the shortest distance and travel time between two points on the road network. In addition, we will visualize the routes that meet the specified criteria (i.e., shortest distance or minimum travel time).<br>

* Shortest distance (Manhattan distance): The shortest distance between two points on a road network, without considering travel speed. <br>
* Minimum travel time: The shortest travel time between two points on a road network, considering travel speed. <br>

### 2.1. Get the nearest nodes of the given locations

In [ ]:
origin_node = emd_gdf.loc[emd_gdf['ADM_NM'] == '회기동', 'nearest_nodes'].values[0]

dest_node = emd_gdf.loc[emd_gdf['ADM_NM'] == '여의동', 'nearest_nodes'].values[0]

print(f"Origin (회기동) Node ID: {origin_node}")
print(f"Destination (여의동) Node ID: {dest_node}")

### 2.2. Calculate the shortest distance

We can use the `shortest_path_length` function from NetworkX to calculate the shortest distance between two nodes in the road network. This function computes the shortest path length using Dijkstra’s algorithm by default.

To perform the calculation, we need to specify the attribute that represents the distance for each edge. In this case, we can use the `length` attribute or `travel_time` attribute.

Syntax: 
```python
    nx.shortest_path_length(G=`NETWORK`, 
                            source=`NEAREST NODE OF ORIGIN`, 
                            target=`NEAREST NODE OF DESTINATION`, 
                            weight=`ATTRIBUTE NAME TO BE WEIGHTED`)
```

Source: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path_length.html

In [ ]:
# Calculate the shortest path with length as the weight
shortest_length = nx.shortest_path_length(G, origin_node, dest_node, weight='length')
shortest_time = nx.shortest_path_length(G, origin_node, dest_node, weight='travel_time')

print(f"Shortest path (length) from {origin_node} to {dest_node}: {shortest_length} Meters")
print(f"Shortest path (travel_time) from {origin_node} to {dest_node}: {shortest_time} Seconds = {shortest_time/60} Minutes")

### 2.3. Visualize the shortest path

We can use the `shortest_path` function from NetworkX to retrieve the actual sequence of nodes that make up the shortest path between two points in the road network.  
Like `shortest_path_length`, this function uses Dijkstra’s algorithm by default when a weighted graph is provided.

To use this function, we need to specify the graph, the origin and destination nodes, and the edge attribute to be used as the weight (e.g., `length` or `travel_time`).

Syntax: 
```python
    nx.shortest_path(G=`NETWORK`,  
                    source=`NEAREST NODE OF ORIGIN`,  
                    target=`NEAREST NODE OF DESTINATION`,  
                    weight=`ATTRIBUTE NAME TO BE WEIGHTED`)
```

Source: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path_length.html


In [ ]:
# returns a list of nodes
route_time = nx.shortest_path(G, origin_node, dest_node, weight='travel_time') 
route_time

In [ ]:
# You can visualize the route using the OSMnx plot function
ox.plot_graph_route(G, route_time, route_linewidth=4, node_size=0, bgcolor='white')

But it might be better to visualize the route using Matplotlib or Folium. <br>
To do this, we need to extract the nodes and edges from the original road network, select the necessary information, and then plot the route with either Matplotlib or Folium.


In [ ]:
# Extract the edges and nodes of the network (graph) into GeoDataFrame with ox.graph_to_gdfs()
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
nodes.head(3)

In [ ]:
# Get the edges based on the u and v node IDs in the route
route_time = nx.shortest_path(G, origin_node, dest_node, weight='travel_time') # Get the route as a list of nodes
od_list_time = [[route_time[i], route_time[i+1]] for i in range(len(route_time)-1)] # Make the nodes as a list of pairs
route_time_gdf = edges.loc[[(u, v, 0) for u, v in od_list_time]] # Select the edges based on the u and v node IDs in the route

route_time_gdf.head(3)

In [ ]:
route_time

In [ ]:
od_list_time

In [ ]:
edges.loc[[(u, v, 0) for u, v in od_list_time]]

In [ ]:
# Same procesure for the shortest distance
route_dist = nx.shortest_path(G, origin_node, dest_node, weight='length') # returns a list of nodes
od_list_dist = [[route_dist[i], route_dist[i+1]] for i in range(len(route_dist)-1)] # 
route_dist_gdf = edges.loc[[(u, v, 0) for u, v in od_list_dist]]

In [ ]:
# Compare the results with Matplotlib
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
edges.plot(ax=ax, color='lightgrey', linewidth=0.5, zorder=1)

nodes.loc[[origin_node]].plot(ax=ax, color='red', markersize=50)
nodes.loc[[dest_node]].plot(ax=ax, color='red', markersize=50)

route_dist_gdf.plot(ax=ax, color='black', linewidth=2, zorder=2) # Shortest distance
route_time_gdf.plot(ax=ax, color='blue', linewidth=2, zorder=2)  # Minimum travel time

plt.show()

In [ ]:
# Same information with Folium
m = folium.Map(location=[37.551, 126.991], zoom_start=12)

route_time_gdf.explore(style_kwds={"color": "blue", "weight": 5}, m=m, name="Shortest Path (Travel Time)")
route_dist_gdf.explore(style_kwds={"color": "black", "weight": 5}, m=m, name="Shortest Path (Distance)")

# Add layer control to the map
folium.LayerControl().add_to(m)

m

## 3. Calculate the accessible area from a given point

Since calculating the shortest path between two points is not scalable, we can use the `single_source_dijkstra_path_length` function from NetworkX to compute the accessible area from a given point all at once. This function computes the shortest path lengths from a source node to all other nodes in the graph using Dijkstra’s algorithm. It returns a dictionary with the target nodes as keys and their corresponding shortest path lengths as values.

Syntax: 
```python
    nx.single_source_dijkstra_path(G=`NETWORK`,  
                                   source=`NEAREST NODE OF ORIGIN`,  
                                   cutoff=`THRESHOLD TIME/DISTANCE`,  
                                   weight=`ATTRIBUTE NAME TO BE WEIGHTED`
                                   )
```

Source: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.weighted.single_source_dijkstra_path_length.html

### 3.1. Calculate the accessible node

In [ ]:
# Calculate the accessible nodes within 10 minutes from the origin node (회기동)
# The return has the key as the node ID and the value as the travel time
access_node_dic = nx.single_source_dijkstra_path_length(G, 
                                                        origin_node, 
                                                        cutoff=1800, # 30 * 60 seconds (30 minutes) 
                                                        weight='travel_time'
                                                        )
access_node_dic

In [ ]:
# Get the keys of the dictionary from access_node_dic
access_node_dic.keys()

In [ ]:
# Check whether the nodes are in the keys of the dictionary
nodes.index.isin(list(access_node_dic.keys()))

In [ ]:
# Select the nodes that are within 600 seconds from the source node
access_node_gdf = nodes.loc[nodes.index.isin(list(access_node_dic.keys()))]
access_node_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

access_node_gdf.plot(ax=ax, color='#0F52BA', markersize=3, zorder=3, alpha=0.7)  # Nodes within 600 secons from the source node
nodes.plot(ax=ax, color='grey', markersize=1, zorder=2)  # All nodes
edges.plot(ax=ax, color='lightgrey', linewidth=1, zorder=1)  # All edges

emd_gdf.loc[emd_gdf['nearest_nodes'] == origin_node].boundary.plot(ax=ax, color='red', linewidth=5, zorder=3)  # Origin node
emd_gdf.boundary.plot(ax=ax, color='black', linewidth=0.5, zorder=1)  # Administrative boundaries

xmin, ymin, xmax, ymax = access_node_gdf.unary_union.bounds

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.get_xaxis().set_visible(False)  # Remove ticks and labels
ax.get_yaxis().set_visible(False)  # Remove ticks and labels

plt.show()

### 3.2. Calculate the accessible area

To get the accessible area, you can simply select the administrative boundary that contains the accessible nodes. <br>
Again, the return of `single_source_dijkstra_path_length` is a dictionary, so you can use the `keys()` method to get the list of accessible nodes. <br>

In [ ]:
access_emd_gdf = emd_gdf.loc[emd_gdf['nearest_nodes'].isin(list(access_node_dic.keys()))]
access_emd_gdf.head(3)

In [ ]:
# Again, the return of `single_source_dijkstra_path_length` is a dictionary.
access_node_dic

In [ ]:
# Compute the travel time in minutes per emd
access_emd_gdf['travel_time'] = access_emd_gdf.apply(lambda x: access_node_dic[x['nearest_nodes']] / 60, axis=1)
access_emd_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

access_emd_gdf.plot(column='travel_time', 
                    cmap='Blues_r', 
                    scheme='UserDefined',
                    classification_kwds={'bins': [0, 5, 10, 15, 20, 25, 30]},
                    legend=True, 
                    edgecolor='black',
                    ax=ax)

ax.get_xaxis().set_visible(False)  # Remove ticks and labels
ax.get_yaxis().set_visible(False)  # Remove ticks and labels

plt.show()

### 3.3. Make the function to calculate the accessible area

In [ ]:
def get_accessible_area_and_time(_network, _gdf, _source_node, _threshold, _weight='travel_time'):
    """
    Get the accessible area from a given node within a specified cutoff distance or time.
    
    Parameters:
    _network (networkx.Graph): The road network graph.
    _gdf (GeoDataFrame): The GeoDataFrame containing the administrative boundaries and nearest nodes.
    _source_node (int): The nearest node to the origin point.
    _threshold (float): The threshold distance or time for accessibility.
    _weight (str): The attribute name to be used as the weight (e.g., 'length' or 'travel_time').
    
    Returns:
    list: A GeoDataFrame which contains the accessible areas within the cutoff distance along with the travel time.
    """
    # Calculate the accessible nodes
    _access_nodes = nx.single_source_dijkstra_path_length(_network, source=_source_node, cutoff=_threshold, weight=_weight)
    
    # Select the accessible area with the accessible nodes
    _access_gdf = _gdf.loc[_gdf['nearest_nodes'].isin(list(_access_nodes.keys()))]

    # Calculate the travel time in minutes per administrative boundary
    _access_gdf['travel_time'] = _access_gdf.apply(lambda x: _access_nodes[x['nearest_nodes']] / 60, axis=1)

    return _access_gdf

In [ ]:
test_gdf = get_accessible_area_and_time(_network=G, 
                                        _gdf=emd_gdf, 
                                        _source_node=origin_node, 
                                        _threshold=1800, 
                                        _weight='travel_time')
test_gdf.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

test_gdf.plot(column='travel_time', 
                    cmap='Blues_r', 
                    scheme='UserDefined',
                    classification_kwds={'bins': [0, 5, 10, 15, 20, 25, 30]},
                    legend=True, 
                    edgecolor='black',
                    ax=ax)

ax.get_xaxis().set_visible(False)  # Remove ticks and labels
ax.get_yaxis().set_visible(False)  # Remove ticks and labels

plt.show()

# Done